In [1]:
from gensim import corpora, models, similarities, matutils
import numpy as np
import os 

# Latent Dirichlet Allocation

LSI pozwala w pewnym sensie znajdowac kluczowe tematy w tekście i dla danego dokumentu określać najbliższy mu temat. Jest to podejście geometryczne. 

LDA jest podejściem probabilistycznym do modelowania tematów. Jest bardziej dokładny choć wolniejszy.

# Model LDA - Latent Dirichlet Allocation (ukryta alokacja Dirichleta)


Motywacja: przedstawienie tekstu jako mieszanki tematów.


Temat - rozkład prawdopodobieństwa na zbiorze słów.


Przykład:
*  <s>Mam</s> gorączkę <s>i</s> katar.
* Graliśmy <s>w</s> siatkówkę.
* Grając <s>w</s> piłkę, wzmacniamy organizm.


Ile "tematów" widzimy?
<br>

<br>

<br>

<br>

<br>

<br>

Intuicyjnie: dwa tematy: "sport" oraz "zdrowie".
* Pierwsze zdanie = 100% zdrowie
* Drugie zdanie = 100% sport
* Trzecie zdanie = 50% sport + 50% zdrowie

LDA to model probabilistyczny, który wykorzystuje dwie wartości prawdopodobieństwa: 

* P (słowo | tematy) 
* P (tematy | dokumenty) 

do stworzenia klastrów.


Proces budowy modelu LDA zakłada więc:
* identyfikację tematów reprezentowanych przez dokumenty korpusu,
* oszacowanie rozkładu prawdopodobieństw wystąpienia wyrazów dla każdego z tematów (stosowany jest rozkład Dirichleta),
* oszacowanie rozkładu prawdopodobieństw wystąpienia tematów w rozpatrywanych dokumentach (stosowany jest również rozkład Dirichleta).

Wykorzystując metodę identyfikacji słów kluczowych opartą na ukrytej alokacji Dirichleta można:
* dla każdego dokumentu obliczyć prawdopodobieństwo wystąpienia w nim każdego z tematów;
* dla każdego tematu obliczyć prawdopodobieństwo wystąpienia słów;
* obliczyć prawdopodobieństwa wystąpienia poszczególnych słów w dokumencie (jako iloczyny dwóch wyżej wymienionych prawdopodobieństw) 

# Generatywność

Mając prawdopodobieństwa $p_1 = P (słowo | tematy) $, $p_2 = P (tematy | dokumenty)$ możemy wygeneraować dokument:

- wybieramy (losujemy) prawdopodobieństwo przynależności dokumentu do tematu (z $p_1$) - dokuemnt zawsze należy do wielu tematów

- generujemy słowa w dokumencie - najpierw losujemy do jakiego tematu należy słowo, a potem generujemy słowo z tego tematu (z $p_2$)

# Uczenie

1. Idziemy przez wszystkie słowa i wszystkie dokumentu i losowo przyporządkowujemy je do tematów.
2. Iteracyjnie idzemy: bierzemy dokuemnt $d$  i słowo $w$ i przyporządkowujemy je do najlepiej pasującego tematu - maksymalizującego $P (słowo | tematy) 
* P (tematy | dokumenty) $
3. Po przejsciu przez wszystkie dokumenty aktualizujemy przydział słów do tematów i tematów do dokumentów i wracamy do punktu 2

# Zad

Na początek wczytujemy korpus z dysku. Użyjemy przykład stworzonego w poprzednim notebook-u.

In [2]:
# documents = ["Romeo and Juliet",
#          "Juliet: O happy dagger",
#          "Romeo died by dagger",
#          "'Live free or die', that’s the New-Hampshire’s motto",
#          "Did you know, New-Hampshire is in New-England"]

In [3]:
if (os.path.exists("tmp/deerwester.dict")):
    dictionary = corpora.Dictionary.load('tmp/deerwester.dict')
    corpus = corpora.MmCorpus('tmp/deerwester.mm')
    print("Used files generated from first tutorial")
else:
    print("Please run first tutorial to generate data set")



Used files generated from first tutorial


In [4]:
for d in corpus:
    print(d)

[(0, 1.0), (1, 1.0)]
[(2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0)]
[(1, 1.0), (2, 1.0), (6, 1.0), (7, 1.0)]
[(8, 1.0), (9, 1.0), (10, 1.0), (11, 1.0), (12, 1.0), (13, 1.0), (14, 1.0)]
[(15, 1.0), (16, 1.0), (17, 1.0), (18, 1.0), (19, 1.0), (20, 1.0)]


In [5]:
scipy_csc_matrix = matutils.corpus2csc(corpus)
print(scipy_csc_matrix.todense())

[[1. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0.]
 [0. 1. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


In [6]:
print(dictionary.token2id)

{'juliet': 0, 'romeo': 1, 'dagger': 2, 'happy': 3, 'juliet:': 4, 'o': 5, 'by': 6, 'died': 7, "'live": 8, "die',": 9, 'free': 10, 'motto': 11, 'new-hampshire’s': 12, 'or': 13, 'that’s': 14, 'did': 15, 'is': 16, 'know,': 17, 'new-england': 18, 'new-hampshire': 19, 'you': 20}


# Budujemy model LDA
Budujemy model LDA i transformujemy dane

* **num_topics=2** oznacza ilość modelowanych tematów

In [7]:
model = models.LdaModel(corpus, id2word=dictionary, num_topics=2)
corpus_lda = model[corpus] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi
for d in corpus_lda:
    print(d)

[(0, 0.22476293), (1, 0.7752371)]
[(0, 0.14855596), (1, 0.85144407)]
[(0, 0.1534796), (1, 0.84652036)]
[(0, 0.06724484), (1, 0.9327552)]
[(0, 0.9248575), (1, 0.075142525)]


### Dla każdego dokumentu dostajemy prawdopodobieństwo przynależności dokumentu do danego tematu.

Możemy też zobaczyć z czego składają się tematy:

In [8]:
model.show_topics()

[(0,
  '0.072*"you" + 0.072*"is" + 0.071*"know," + 0.071*"new-hampshire" + 0.071*"did" + 0.071*"new-england" + 0.058*"dagger" + 0.054*"romeo" + 0.050*"by" + 0.045*"juliet:"'),
 (1,
  '0.080*"romeo" + 0.077*"dagger" + 0.061*"motto" + 0.061*"or" + 0.060*"\'live" + 0.060*"new-hampshire’s" + 0.060*"that’s" + 0.060*"die\'," + 0.059*"free" + 0.050*"o"')]

In [9]:
model.print_topics(2)

[(0,
  '0.072*"you" + 0.072*"is" + 0.071*"know," + 0.071*"new-hampshire" + 0.071*"did" + 0.071*"new-england" + 0.058*"dagger" + 0.054*"romeo" + 0.050*"by" + 0.045*"juliet:"'),
 (1,
  '0.080*"romeo" + 0.077*"dagger" + 0.061*"motto" + 0.061*"or" + 0.060*"\'live" + 0.060*"new-hampshire’s" + 0.060*"that’s" + 0.060*"die\'," + 0.059*"free" + 0.050*"o"')]

In [10]:
model.print_topics(num_topics=2, num_words=4)

[(0, '0.072*"you" + 0.072*"is" + 0.071*"know," + 0.071*"new-hampshire"'),
 (1, '0.080*"romeo" + 0.077*"dagger" + 0.061*"motto" + 0.061*"or"')]

# Zad

Chcemy posortować słowa każdego tematu i wybrać 5 najważniejszych - co można powiedzieć o tematach?

Proszę zobaczyć na funkcje typu get_topics(), get_term_topics(...): https://radimrehurek.com/gensim/models/ldamodel.html

In [11]:
import numpy as np
topics = np.argsort(model.get_topics()[0,:])[::-1] #::-1 sortowanie w odwrotnej kolejności
for x in topics[:5]:
    print(dictionary[x])

you
is
know,
new-hampshire
did


In [12]:
model.get_topic_terms(topicid=0)

[(20, 0.071840286),
 (16, 0.07153516),
 (17, 0.071404055),
 (19, 0.07138407),
 (15, 0.071238615),
 (18, 0.070652395),
 (2, 0.05751301),
 (1, 0.054392435),
 (6, 0.05003244),
 (4, 0.04501523)]

In [13]:
topic2_terms = model.get_topic_terms(topicid=0)
topic2_words = [
    (dictionary.get(i), j)
    for i,j in topic2_terms
]
topic2_words

[('you', 0.071840286),
 ('is', 0.07153516),
 ('know,', 0.071404055),
 ('new-hampshire', 0.07138407),
 ('did', 0.071238615),
 ('new-england', 0.070652395),
 ('dagger', 0.05751301),
 ('romeo', 0.054392435),
 ('by', 0.05003244),
 ('juliet:', 0.04501523)]

In [14]:
[
    (dictionary.get(i), j)
    for i,j in model.get_topic_terms(topicid=0)
]

[('you', 0.071840286),
 ('is', 0.07153516),
 ('know,', 0.071404055),
 ('new-hampshire', 0.07138407),
 ('did', 0.071238615),
 ('new-england', 0.070652395),
 ('dagger', 0.05751301),
 ('romeo', 0.054392435),
 ('by', 0.05003244),
 ('juliet:', 0.04501523)]

# Zad 

Proszę posortować zdania najbardziej pasujące do danego tematu. 

In [15]:
import gensim

top_inex = 0

numpy_corpus = gensim.matutils.corpus2dense(corpus_lda, num_terms=2)
docs = np.argsort(numpy_corpus[top_inex,:])[::-1]
for x in docs[:5]:
    print(corpus[x])
    
#trzeba by wypisać raczej zdania niż ich reprezentacje bag-of-words, ale tu nie mam dostepu do tekstu

[(15, 1.0), (16, 1.0), (17, 1.0), (18, 1.0), (19, 1.0), (20, 1.0)]
[(0, 1.0), (1, 1.0)]
[(1, 1.0), (2, 1.0), (6, 1.0), (7, 1.0)]
[(2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0)]
[(8, 1.0), (9, 1.0), (10, 1.0), (11, 1.0), (12, 1.0), (13, 1.0), (14, 1.0)]


In [16]:
top_inex = 1

numpy_corpus = gensim.matutils.corpus2dense(corpus_lda, num_terms=2)
docs = np.argsort(numpy_corpus[top_inex,:])[::-1]
for x in docs[:5]:
    print(corpus[x])
    

[(8, 1.0), (9, 1.0), (10, 1.0), (11, 1.0), (12, 1.0), (13, 1.0), (14, 1.0)]
[(2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0)]
[(1, 1.0), (2, 1.0), (6, 1.0), (7, 1.0)]
[(0, 1.0), (1, 1.0)]
[(15, 1.0), (16, 1.0), (17, 1.0), (18, 1.0), (19, 1.0), (20, 1.0)]


In [17]:
documents = ["Romeo and Juliet",
         "Juliet: O happy dagger",
         "Romeo died by dagger",
         "'Live free or die', that’s the New-Hampshire’s motto",
         "Did you know, New-Hampshire is in New-England"]

top_inex = 0

numpy_corpus = gensim.matutils.corpus2dense(corpus_lda, num_terms=2)
docs = np.argsort( numpy_corpus[top_inex,:] )[::-1]
for x in docs[:5]:
    print(documents[x])
    

Did you know, New-Hampshire is in New-England
Romeo and Juliet
Romeo died by dagger
Juliet: O happy dagger
'Live free or die', that’s the New-Hampshire’s motto


In [18]:
top_inex = 1

numpy_corpus = gensim.matutils.corpus2dense(corpus_lda, num_terms=2)
docs = np.argsort(numpy_corpus[top_inex,:])[::-1]
for x in docs[:5]:
    print(documents[x])
    

'Live free or die', that’s the New-Hampshire’s motto
Juliet: O happy dagger
Romeo died by dagger
Romeo and Juliet
Did you know, New-Hampshire is in New-England


# Zad. 
Sprawdzić do jakiego tematu pasuje nowy dokument i jakie są mu najbliższe

In [19]:
doc = "died dagger"

In [20]:
doc_rep = dictionary.doc2bow(doc.split(' '))
# print(doc_rep)
doc_assignments = model[doc_rep]
print(doc_assignments)

[(0, 0.22685523), (1, 0.7731448)]


In [21]:
index = similarities.MatrixSimilarity(corpus_lda)

sims = index[doc_assignments]
print(list(enumerate(sims)))

sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims)

[(0, 0.9999944), (1, 0.99366546), (2, 0.9943736), (3, 0.9773077), (4, 0.35832596)]
[(0, 0.9999944), (2, 0.9943736), (1, 0.99366546), (3, 0.9773077), (4, 0.35832596)]


# Wizualizacja modelu LDA:

pyLDAvis

http://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/pyLDAvis_overview.ipynb


* Czerwone słupki reprezentują częstotliwość słów w danym temacie (proporcjonalnie do $P (słowo \ | \ tematy) $), 
* Niebieskie słupki reprezentują częstotliwość tematów w całym korpusie (proporcjonalnie do $P(tematy \ | \ dokumenty)$. 

Zmień wartość $\lambda$, aby dostosować rankingi słów: 
 * małe wartości $\lambda$ (blisko 0) podkreślają potencjalnie rzadkie, ale ekskluzywne słowa dla wybranego tematu
 * duże wartości $\lambda$ (blisko 1) podkreślają częste, ale niekoniecznie ekskluzywne słowa dla wybranego tematu. 
 
W http://www.kennyshirley.com/LDAvis/ sugeruje się żeby ustawiać $\lambda$ w pobliżu 0.6. Podobno pomaga to użytkownikom w interpretacji tematu.

In [22]:
import pyLDAvis.gensim

In [23]:
pyLDAvis.enable_notebook()

In [24]:
# pyLDAvis.gensim.prepare??

In [35]:
pyLDAvis.gensim.prepare(model, corpus, dictionary)

C:\Users\admin\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.032103  0.0       1        1  55.207668
1     -0.032103  0.0       2        1  44.792328, topic_info=     Category      Freq             Term     Total  loglift  logprob
term                                                                
20    Default  1.000000              you  1.000000  21.0000  21.0000
19    Default  1.000000    new-hampshire  1.000000  20.0000  20.0000
18    Default  1.000000      new-england  1.000000  19.0000  19.0000
15    Default  1.000000              did  1.000000  18.0000  18.0000
17    Default  1.000000            know,  1.000000  17.0000  17.0000
16    Default  1.000000               is  1.000000  16.0000  16.0000
3     Default  1.000000            happy  1.000000  15.0000  15.0000
4     Default  1.000000          juliet:  1.000000  14.0000  14.0000
1     Default  1.000000            romeo  1.000000  13.0000  13.0000
5     Default  1.000000                o  1.000000  12.0000  12.0000
14    Default  1.000000           that’s  1.000000  11.0000  11.0000
13    Default  1.000000               or  1.000000  10.0000  10.0000
8     Default  1.000000            'live  1.000000   9.0000   9.0000
12    Default  1.000000  new-hampshire’s  1.000000   8.0000   8.0000
10    Default  1.000000             free  1.000000   7.0000   7.0000
11    Default  1.000000            motto  1.000000   6.0000   6.0000
9     Default  1.000000            die',  1.000000   5.0000   5.0000
6     Default  1.000000               by  1.000000   4.0000   4.0000
7     Default  1.000000             died  1.000000   3.0000   3.0000
0     Default  1.000000           juliet  1.000000   2.0000   2.0000
2     Default  1.000000           dagger  1.000000   1.0000   1.0000
1      Topic1  1.197517            romeo  1.571662   0.3222  -2.3612
14     Topic1  0.744293           that’s  1.047209   0.2526  -2.8367
13     Topic1  0.740874               or  1.047173   0.2480  -2.8414
8      Topic1  0.737110            'live  1.047133   0.2430  -2.8464
12     Topic1  0.735540  new-hampshire’s  1.047117   0.2409  -2.8486
10     Topic1  0.734251             free  1.047103   0.2391  -2.8503
11     Topic1  0.730762            motto  1.047067   0.2344  -2.8551
9      Topic1  0.729810            die',  1.047057   0.2331  -2.8564
6      Topic1  0.727366               by  1.047031   0.2298  -2.8598
...       ...       ...              ...       ...      ...      ...
5      Topic1  0.448005                o  1.044099  -0.2520  -3.3444
4      Topic1  0.388627          juliet:  1.043476  -0.3936  -3.4866
3      Topic1  0.388315            happy  1.043473  -0.3944  -3.4874
16     Topic1  0.353452               is  1.043107  -0.4881  -3.5814
17     Topic1  0.345266            know,  1.043020  -0.5115  -3.6049
15     Topic1  0.337200              did  1.042936  -0.5350  -3.6285
18     Topic1  0.333303      new-england  1.042896  -0.5466  -3.6401
19     Topic1  0.332410    new-hampshire  1.042886  -0.5493  -3.6428
20     Topic1  0.317265              you  1.042727  -0.5958  -3.6894
20     Topic2  0.725462              you  1.042727   0.4403  -2.6533
19     Topic2  0.710476    new-hampshire  1.042886   0.4193  -2.6742
18     Topic2  0.709593      new-england  1.042896   0.4181  -2.6754
15     Topic2  0.705736              did  1.042936   0.4126  -2.6809
17     Topic2  0.697754            know,  1.043020   0.4011  -2.6922
16     Topic2  0.689655               is  1.043107   0.3894  -2.7039
3      Topic2  0.655158            happy  1.043473   0.3377  -2.7552
4      Topic2  0.654849          juliet:  1.043476   0.3372  -2.7557
5      Topic2  0.596094                o  1.044099   0.2426  -2.8497
2      Topic2  0.622517           dagger  1.569028  -0.1213  -2.8063
0      Topic2  0.332740           juliet  1.046892  -0.3431  -3.4328
7      Topic2  0.331174             died  1.046909  -0.3478  -3.4375
6      Topic2  0.319665            